In [102]:
from PIL import Image
from math import sqrt

In [78]:
img = Image.open("source.jpg").convert("L")


In [94]:
#img.save("source_greyscale.jpg")

In [97]:
def convolutionMask(img,pixels,position,mask):
    valueList = []
    pX = position[0]
    pY = position[1]
    for i in range(3):
        mY = i-1+pY
        for j in range(3):
            mX = j-1 + pX
            if mX < 0 or mX >= img.width or mY < 0 or mY >= img.height:
                valueList.append(pixels[pX,pY] * mask[i][j])
                continue    
                
            value = pixels[mX,mY] * mask[i][j]
            valueList.append(value)

    result= sum(valueList)
    return result

In [98]:
pixels = img.load()
mask = [[-1,-1,-1],
        [-1,8,-1],
        [-1,-1,-1]]
print(convolutionMask(img,pixels,(0,0),mask))
print(pixels[99,99],pixels[99,100],pixels[99,101],pixels[100,99],pixels[100,100],pixels[100,101],pixels[101,99],pixels[101,100],pixels[101,101])
print(convolutionMask(img,pixels,[100,100],mask))

14
73 73 73 75 75 75 75 76 76
4


In [90]:
def createImageEdge(img):
    output =Image.new('L',(img.width,img.height),'white')
    outputPixels = output.load()
    #print(img.width,img.height)
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            value = int(convolutionMask(img,pixels,(i,j),mask)) #+ pixels[i,j][0]
            #print(value)
            outputPixels[i,j] = value
    return output
    


In [91]:
def mergeImage(img1,img2):
    pixels1 = img1.load()
    pixels2 = img2.load()
    
    output = Image.new('L',(img1.width,img1.height),"white")
    outputPixel = output.load()
    for j in range(img1.height):
        for i in range(img1.width):
            value =  pixels1[i,j] + pixels2[i,j]
            outputPixel[i,j] = value
            
    return output
            

In [99]:
edgeImg = createImageEdge(img)
edgeImg.save("process1.jpg")

In [100]:
mergeImg = mergeImage(img,edgeImg)
mergeImg.save('process2.jpg')

In [109]:
def createSobelFilterImage(img):
    maskX = [[-1,0,1],
             [-2,0,2],
             [-1,0,1]]
    maskY = [[-1,-2,-1],
             [0,0,0],
             [1,2,1]]
    
    output = Image.new('L',(img.width,img.height),"white")
    outputPixel = output.load()
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            gX = convolutionMask(img,pixels,(i,j),maskX)
            gY = convolutionMask(img,pixels,(i,j),maskY)
            value = int(sqrt(gX*gX + gY*gY))
            outputPixel[i,j] = value
    return output

In [110]:
sobelImg = createSobelFilterImage(img)
sobelImg.save('process3.jpg')

In [120]:
def advangeImage(img):
    mask = [[0.1111,0.1111,0.1111],
        [0.1111,0.1111,0.1111],
        [0.1111,0.1111,0.1111]]
    output = Image.new('L',(img.width,img.height),"white")
    outputPixel = output.load()
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            value = convolutionMask(img,pixels,(i,j),mask)
            outputPixel[i,j] = int(value)
    return output

In [121]:
advancedImg = advangeImage(sobelImg)
advancedImg.save('process4.jpg')

In [117]:
def normalizeAndMerge(normalizeImg,inputImg):
    pixelsNormal = normalizeImg.load()
    inputPixels = inputImg.load()
    
    output = Image.new('L',(normalizeImg.width,normalizeImg.height),"white")
    outputPixel = output.load()
    
    for j in range(img.height):
        for i in range(img.width):
            value = pixelsNormal[i,j] / 255 * inputPixels[i,j]
            outputPixel[i,j] = int(value)
    return output
    

In [122]:
noramlizedImg = normalizeAndMerge(advancedImg,edgeImg)
noramlizedImg.save('process5.jpg')

In [123]:
finalImg = mergeImage(noramlizedImg,img)
finalImg.save('process6.jpg')